In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

In [57]:
#Loading inflation data 


if_df = pd.read_csv('scraped_cpi.csv')

In [69]:
print(if_df.columns)

Index(['base_year', 'series', 'year', 'month', 'state', 'sector', 'division',
       'group', 'class', 'sub_class', 'item', 'code', 'index', 'inflation',
       'imputation'],
      dtype='object')


In [59]:
# Debug: Check for complete duplicate rows in the original dataframe BEFORE MoM calculation
print("=" * 80)
print("DUPLICATE ROWS ANALYSIS (ORIGINAL DATA)")
print("=" * 80)

print(f"\nTotal rows in if_df: {len(if_df)}")

# Check for complete duplicates (all columns match)
duplicate_mask = if_df.duplicated(keep=False)
complete_duplicates = if_df[duplicate_mask]

print(f"\nTotal complete duplicate rows (including originals): {len(complete_duplicates)}")

# Show duplicates grouped by their duplicate groups
print("\n" + "-" * 80)
print("DUPLICATE GROUPS:")
print("-" * 80)

if len(complete_duplicates) > 0:
    # Group by all columns to find exact duplicates
    duplicate_groups = if_df[duplicate_mask].groupby(list(if_df.columns), dropna=False)
    
    print(f"\nFound {len(duplicate_groups)} unique duplicate groups:\n")
    
    for i, (name, group) in enumerate(duplicate_groups, 1):
        print(f"Duplicate Group #{i} - ({len(group)} identical rows):")
        print(group.to_string())
        print()
else:
    print("✓ No complete duplicate rows found!")

# Summary statistics BEFORE removal
print("\n" + "=" * 80)
print("SUMMARY STATISTICS (BEFORE REMOVAL):")
print("=" * 80)
print(f"Total rows: {len(if_df)}")
print(f"Duplicate rows: {len(complete_duplicates)}")
print(f"Percentage of duplicates: {(len(complete_duplicates) / len(if_df) * 100):.2f}%")
print(f"Unique rows: {len(if_df.drop_duplicates())}")

# Remove duplicates, keeping only the first instance
if_df_cleaned = if_df.drop_duplicates(keep='first').reset_index(drop=True)

# Summary statistics AFTER removal
print("\n" + "=" * 80)
print("SUMMARY STATISTICS (AFTER REMOVAL):")
print("=" * 80)
print(f"Total rows remaining: {len(if_df_cleaned)}")
print(f"Rows deleted: {len(if_df) - len(if_df_cleaned)}")
print(f"Percentage removed: {((len(if_df) - len(if_df_cleaned)) / len(if_df) * 100):.2f}%")

print("\n" + "=" * 80)
print("ACTION TAKEN:")
print("=" * 80)
print(f"✓ Removed {len(if_df) - len(if_df_cleaned)} duplicate rows")
print(f"✓ Kept first instance of each duplicate group")
print(f"✓ Updated if_df with cleaned data")
print("=" * 80)

# Update if_df with the cleaned version
if_df = if_df_cleaned

DUPLICATE ROWS ANALYSIS (ORIGINAL DATA)

Total rows in if_df: 8684

Total complete duplicate rows (including originals): 566

--------------------------------------------------------------------------------
DUPLICATE GROUPS:
--------------------------------------------------------------------------------

Found 200 unique duplicate groups:

Duplicate Group #1 - (3 identical rows):
      base_year   series  year   month      state    sector                                                            division                                      group                                     class                            sub_class                                    item           code   index inflation imputation
5862       2024  Current  2025  August  All India  Combined  Furnishings, household equipment and routine household maintenance  Furniture, furnishings, and loose carpets  Furniture, furnishings and loose carpets  Furnishings, loose carpets and rugs  Carpet, daree and other floor m

In [66]:
def calculate_mom_change(df, value_column='index', group_columns=None, new_column_name='mom_change'):
    """
    Calculate month-on-month percentage change for inflation data.

    Parameters:
    df (pd.DataFrame): Input dataframe with inflation data
    value_column (str): Column name to calculate change on (default: 'index')
    group_columns (list): Columns to group by for unique series (default: hierarchical categories)
    new_column_name (str): Name for the new MoM change column (default: 'mom_change')

    Returns:
    pd.DataFrame: DataFrame with added MoM change column and date column
    """
    if group_columns is None:
        group_columns = ['state', 'sector', 'division', 'group', 'class', 'sub_class', 'item', 'code']

    # Create a copy to avoid modifying original
    df_copy = df.copy()

    # Create a proper datetime column for sorting
    # Combine year and month into a date string and parse it
    df_copy['date'] = pd.to_datetime(df_copy['year'].astype(str) + '-' + df_copy['month'], format='%Y-%B')

    # Sort the dataframe by group columns and chronological date
    sort_columns = group_columns + ['date']
    df_sorted = df_copy.sort_values(sort_columns).copy()

    # Calculate percentage change within each group
    df_sorted[new_column_name] = df_sorted.groupby(group_columns)[value_column].pct_change() * 100

    return df_sorted

def calculate_yoy_change(df, value_column='index', group_columns=None, new_column_name='yoy_change'):
    """
    Calculate year-on-year percentage change for inflation data.

    Parameters:
    df (pd.DataFrame): Input dataframe with inflation data
    value_column (str): Column name to calculate change on (default: 'index')
    group_columns (list): Columns to group by for unique series (default: hierarchical categories)
    new_column_name (str): Name for the new YoY change column (default: 'yoy_change')

    Returns:
    pd.DataFrame: DataFrame with added YoY change column
    """
    if group_columns is None:
        group_columns = ['state', 'sector', 'division', 'group', 'class', 'sub_class', 'item', 'code']

    # Create a copy to avoid modifying original
    df_copy = df.copy()

    # Create a proper datetime column for sorting if it doesn't exist
    if 'date' not in df_copy.columns:
        df_copy['date'] = pd.to_datetime(df_copy['year'].astype(str) + '-' + df_copy['month'], format='%Y-%B')

    # Sort the dataframe by group columns and chronological date
    sort_columns = group_columns + ['date']
    df_sorted = df_copy.sort_values(sort_columns).copy()

    # Calculate percentage change with a period of 12 (months) within each group
    df_sorted[new_column_name] = df_sorted.groupby(group_columns)[value_column].pct_change(periods=12) * 100

    return df_sorted

In [67]:
# Apply month-on-month and year-on-year change calculations
if_df_mom = calculate_mom_change(if_df, value_column='index')
# Pass the dataframe with MoM already calculated to add YoY column to it
if_df_analysis = calculate_yoy_change(if_df_mom, value_column='index')

# Display sample of the results with proper chronological order
print("Sample of data with MoM and YoY change (chronologically sorted):")
sample = if_df_analysis[(if_df_analysis['state'] == 'All India') & (if_df_analysis['item'] == '*')].tail(20)
print(sample[['date', 'year', 'month', 'index', 'mom_change', 'yoy_change']])

Sample of data with MoM and YoY change (chronologically sorted):
           date  year      month   index  mom_change  yoy_change
4615 2025-07-01  2025       July  100.23    0.049910         NaN
5283 2025-08-01  2025     August  100.25    0.019954         NaN
5871 2025-09-01  2025  September  100.27    0.019950         NaN
6499 2025-10-01  2025    October  107.24    6.951232         NaN
7137 2025-11-01  2025   November  107.41    0.158523         NaN
7765 2025-12-01  2025   December  107.49    0.074481         NaN
115  2026-01-01  2026    January  107.61    0.111638         NaN
942  2025-01-01  2025    January  100.12         NaN         NaN
1570 2025-02-01  2025   February  100.14    0.019976         NaN
2198 2025-03-01  2025      March  100.15    0.009986         NaN
2866 2025-04-01  2025      April  100.16    0.009985         NaN
3534 2025-05-01  2025        May  100.18    0.019968         NaN
4146 2025-06-01  2025       June  100.19    0.009982         NaN
4774 2025-07-01  2025    

In [ ]:
# Save the final analysis data with custom indexes
if_df_with_custom.to_csv('inflation_analysis_results_with_exclusions.csv', index=False)
print("✓ Saved final analysis data including custom exclusion indexes to 'inflation_analysis_results_with_exclusions.csv'")

✓ Saved final analysis data with MoM and YoY changes to 'inflation_analysis_results.csv'


In [70]:
# Inspection of unique categories to map weights correctly
print("Unique Divisions in data:")
print(if_df_analysis['division'].unique())

print("\nUnique Groups in data (top 20):")
print(if_df_analysis['group'].unique()[:20])

# Load weights for calculation
divisions_weights = pd.read_csv('../weights_new/divisions.csv')
groups_weights = pd.read_csv('../weights_new/groups.csv')

print("\nDivisions Weights columns:", divisions_weights.columns.tolist())
print("Groups Weights columns:", groups_weights.columns.tolist())

Unique Divisions in data:
['CPI (General)' 'Clothing and footwear' 'Education services'
 'Food and beverages'
 'Furnishings, household equipment and routine household maintenance'
 'Health' 'Housing, water, electricity, gas and other fuels'
 'Information and communication' 'Paan, tobacco and intoxicants'
 'Personal care, social protection and miscellaneous goods and services'
 'Recreation, sport and culture' 'Restaurants and accommodation services'
 'Transport']

Unique Groups in data (top 20):
['*' 'Clothing' 'Footwear' 'Early childhood and primary education'
 'Education not defined by level' 'Higher education' 'Secondary education'
 'Beverages' 'Food' 'Services for processing primary goods for food'
 'Furniture, furnishings, and loose carpets'
 'Glassware, tableware and household utensils'
 'Goods and services for routine household maintenance'
 'Household appliances' 'Household textiles'
 'Tools and equipment for house and garden' 'Inpatient care services'
 'Medicines and health pro

In [72]:
# Load all weights for manual index calculations
weights_path = '../weights_new/'
divisions_w = pd.read_csv(f'{weights_path}divisions.csv')
groups_w = pd.read_csv(f'{weights_path}groups.csv')
classes_w = pd.read_csv(f'{weights_path}classes.csv')
subclasses_w = pd.read_csv(f'{weights_path}subclasses.csv')
items_w = pd.read_csv(f'{weights_path}items.csv')

# Clean up Column Names in weights to match data if necessary
# Generally, we want to align 'Division_Name' with 'division' etc.
divisions_w['Division_Name'] = divisions_w['Division_Name'].str.strip()

print("Weights loaded successfully.")
print(f"Divisions: {len(divisions_w)}, Groups: {len(groups_w)}, Classes: {len(classes_w)}, Items: {len(items_w)}")

Weights loaded successfully.
Divisions: 12, Groups: 43, Classes: 92, Items: 358


In [73]:
# Find groups for Fuel and Food
print("Groups containing 'Fuel':")
print(groups_w[groups_w['Group_Name'].str.contains('Fuel', case=False)]['Group_Name'].tolist())

print("\nGroups containing 'Food':")
print(groups_w[groups_w['Group_Name'].str.contains('Food', case=False)]['Group_Name'].tolist())

# Check total weight of these
fuel_groups = groups_w[groups_w['Group_Name'].str.contains('Fuel', case=False)]
food_groups = groups_w[groups_w['Group_Name'].str.contains('Food', case=False)]

print(f"\nFuel Total Weight: {fuel_groups['Weight'].sum():.2f}")
print(f"Food Total Weight: {food_groups['Weight'].sum():.2f}")

Groups containing 'Fuel':
['Electricity, gas and other fuels']

Groups containing 'Food':
['Food', 'Services for processing primary goods for food', 'Food and beverage serving services']

Fuel Total Weight: 5.36
Food Total Weight: 38.31


In [74]:
def get_custom_cpi_index(df, level_column, exclusions, weights_df, weight_col_name, weight_val_name, index_name):
    """
    Calculates a custom CPI index by excluding certain components at a specific level.
    
    Parameters:
    df (pd.DataFrame): The main inflation dataframe (if_df_analysis)
    level_column (str): The column in df to filter on (e.g., 'division', 'group', 'class')
    exclusions (list): List of values in level_column to exclude
    weights_df (pd.DataFrame): The weights dataframe corresponding to that level
    weight_col_name (str): The column name in weights_df that matches level_column
    weight_val_name (str): The column name for weight values in weights_df (usually 'Weight')
    index_name (str): The name to give to this new index
    
    Returns:
    pd.DataFrame: A dataframe with the new index series
    """
    
    # 1. Filter weights to exclude specified components
    remaining_weights = weights_df[~weights_df[weight_col_name].isin(exclusions)].copy()
    total_remaining_weight = remaining_weights[weight_val_name].sum()
    
    # Pre-normalize weights to sum to 100 for simplicity (though weighted average handles it)
    remaining_weights['normalized_weight'] = (remaining_weights[weight_val_name] / total_remaining_weight) * 100
    
    # 2. Filter the main dataframe to include only the remaining components at the correct level
    # We need to ensure we're at the right level (e.g., items within that level should be '*')
    # However, the user might want to exclude at group level while we calculate from division level.
    # The safest way is to go to the level specified.
    
    # For a specific level (like group), higher levels should be specific, and lower levels should be '*'
    # But usually, if we group by the level_column, it's sufficient if we filter for the right level rows.
    
    # Get the rows for the components we want to include
    # Using '*' for lower levels to ensure we are using the aggregated values for those components
    mask = (df[level_column].isin(remaining_weights[weight_col_name]))
    
    # We need to be careful: if we are at 'group' level, 'class', 'sub_class', 'item' should be '*'
    # Let's find what lower levels exist
    all_levels = ['division', 'group', 'class', 'sub_class', 'item']
    try:
        current_idx = all_levels.index(level_column)
        lower_levels = all_levels[current_idx + 1:]
        for lvl in lower_levels:
            mask = mask & (df[lvl] == '*')
    except ValueError:
        pass
        
    filtered_df = df[mask].copy()
    
    # 3. Join with weights
    merged = filtered_df.merge(
        remaining_weights[[weight_col_name, 'normalized_weight']], 
        left_on=level_column, 
        right_on=weight_col_name
    )
    
    # 4. Calculate weighted index for each date/state/sector
    group_cols = ['date', 'year', 'month', 'state', 'sector']
    
    def weighted_avg(group):
        return (group['index'] * group['normalized_weight']).sum() / group['normalized_weight'].sum()
    
    custom_index = merged.groupby(group_cols).apply(weighted_avg).reset_index()
    custom_index.columns = group_cols + ['index']
    
    # 5. Add metadata to match original structure
    custom_index['division'] = index_name
    for lvl in ['group', 'class', 'sub_class', 'item', 'code']:
        custom_index[lvl] = '*'
        
    return custom_index

# Example: Calculate Core CPI (Excluding Food & Beverages and Fuel)
# Note: In India, Core is often defined as CPI ex (Food, Fuel & Light)

# 1. Traditional Core (Ex Food & Beverages)
cpi_ex_food = get_custom_cpi_index(
    if_df_analysis, 
    level_column='division', 
    exclusions=['Food and beverages'],
    weights_df=divisions_w,
    weight_col_name='Division_Name',
    weight_val_name='Weight',
    index_name='CPI (Ex Food & Beverages)'
)

# 2. Food & Fuel Exclusion (Approximate)
# Food is a division, Fuel is a group within 'Housing...'
# To do this cleanly, we should work at the GROUP level for EVERYTHING.

# First, let's prepare group level data
cpi_ex_food_fuel = get_custom_cpi_index(
    if_df_analysis,
    level_column='group',
    exclusions=['Food', 'Services for processing primary goods for food', 'Electricity, gas and other fuels'],
    weights_df=groups_w,
    weight_col_name='Group_Name',
    weight_val_name='Weight',
    index_name='CPI (Core - Ex Food & Fuel)'
)

print("Custom indexes calculated.")
print(f"Ex-Food samples:\n{cpi_ex_food.tail(3)}")
print(f"\nCore-Ex-Food-Fuel samples:\n{cpi_ex_food_fuel.tail(3)}")


Custom indexes calculated.
Ex-Food samples:
         date  year     month      state    sector       index  \
10 2025-11-01  2025  November  All India  Combined  103.778590   
11 2025-12-01  2025  December  All India  Combined  104.108920   
12 2026-01-01  2026   January  All India  Combined  104.717158   

                     division group class sub_class item code  
10  CPI (Ex Food & Beverages)     *     *         *    *    *  
11  CPI (Ex Food & Beverages)     *     *         *    *    *  
12  CPI (Ex Food & Beverages)     *     *         *    *    *  

Core-Ex-Food-Fuel samples:
         date  year     month      state    sector       index  \
10 2025-11-01  2025  November  All India  Combined  104.055145   
11 2025-12-01  2025  December  All India  Combined  104.394105   
12 2026-01-01  2026   January  All India  Combined  105.041398   

                       division group class sub_class item code  
10  CPI (Core - Ex Food & Fuel)     *     *         *    *    *  
11  CPI (C

In [75]:
# Define common exclusions
custom_indexes = []

# 1. Headline General Index (Already in data, but for completeness)
# 2. Ex-Food and Beverages
custom_indexes.append(get_custom_cpi_index(
    if_df_analysis, 'division', ['Food and beverages'], 
    divisions_w, 'Division_Name', 'Weight', 'CPI Ex-Food-Beverages'
))

# 3. Core CPI (Ex Food & Fuel)
# Fuel is usually 'Electricity, gas and other fuels' group
custom_indexes.append(get_custom_cpi_index(
    if_df_analysis, 'group', ['Food', 'Services for processing primary goods for food', 'Electricity, gas and other fuels'], 
    groups_w, 'Group_Name', 'Weight', 'CPI Core (Ex-Food-Fuel)'
))

# 4. Ex-Vegetables
# First find the actual name/level for Vegetables
veg_match = classes_w[classes_w['Class_Name'].str.contains('Vegetables', case=False)]
if not veg_match.empty:
    veg_name = veg_match.iloc[0]['Class_Name']
    custom_indexes.append(get_custom_cpi_index(
        if_df_analysis, 'class', [veg_name], 
        classes_w, 'Class_Name', 'Weight', 'CPI Ex-Vegetables'
    ))

# Combine all custom indexes
custom_df = pd.concat(custom_indexes, ignore_index=True)

# Apply MoM and YoY calculations to the custom indexes
custom_df = calculate_mom_change(custom_df, value_column='index', group_columns=['division'])
custom_df = calculate_yoy_change(custom_df, value_column='index', group_columns=['division'])

# Append to the main analysis dataframe for comparison
if_df_with_custom = pd.concat([if_df_analysis, custom_df], ignore_index=True)

print("Added custom exclusion indexes to the analysis dataframe.")
print("\nComparing YoY Inflation (January 2026):")
latest_date = if_df_with_custom['date'].max()
comparison = if_df_with_custom[
    (if_df_with_custom['date'] == latest_date) & 
    (if_df_with_custom['state'] == 'All India') &
    (if_df_with_custom['sector'] == 'Combined') &
    (if_df_with_custom['group'] == '*') &
    (if_df_with_custom['division'].isin(['CPI (General)', 'CPI Ex-Food-Beverages', 'CPI Core (Ex-Food-Fuel)', 'CPI Ex-Vegetables']))
]
print(comparison[['division', 'index', 'mom_change', 'yoy_change']])


Added custom exclusion indexes to the analysis dataframe.

Comparing YoY Inflation (January 2026):
                     division       index  mom_change  yoy_change
12              CPI (General)  104.460000    0.345821    2.744172
8330  CPI Core (Ex-Food-Fuel)  105.041398    0.620047    3.320203
8343    CPI Ex-Food-Beverages  104.717158    0.584233    3.121484
8356        CPI Ex-Vegetables  104.008771    0.178648    1.479363
